In [1]:
import pandas as pd

penguins = pd.read_csv("../datasets/penguins.csv")

columns = ["Body Mass (g)", "Flipper Length (mm)", "Culmen Length (mm)"]
target_name = "Species"

# Remove lines with missing values for the columns of interest
penguins_non_missing = penguins[columns + [target_name]].dropna()

data = penguins_non_missing[columns]
target = penguins_non_missing[target_name]

## Question 1
Inspect the target variable and select the correct assertions from the following proposals.

In [2]:
target.unique()

array(['Adelie Penguin (Pygoscelis adeliae)',
       'Gentoo penguin (Pygoscelis papua)',
       'Chinstrap penguin (Pygoscelis antarctica)'], dtype=object)

In [3]:
target.value_counts()

Adelie Penguin (Pygoscelis adeliae)          151
Gentoo penguin (Pygoscelis papua)            123
Chinstrap penguin (Pygoscelis antarctica)     68
Name: Species, dtype: int64

c) The problem to be solved is a multiclass classification problem (more than 2 possible classes

## Question 2
Inspect the statistics of the target and individual features to select the correct statements.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342 entries, 0 to 343
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Body Mass (g)        342 non-null    float64
 1   Flipper Length (mm)  342 non-null    float64
 2   Culmen Length (mm)   342 non-null    float64
dtypes: float64(3)
memory usage: 10.7 KB


In [5]:
data.describe()

,Body Mass (g),Flipper Length (mm),Culmen Length (mm)
count,342.000000,342.000000,342.000000
mean,4201.754386,200.915205,43.921930
std,801.954536,14.061714,5.459584
min,2700.000000,172.000000,32.100000
25%,3550.000000,190.000000,39.225000
50%,4050.000000,197.000000,44.450000
75%,4750.000000,213.000000,48.500000
max,6300.000000,231.000000,59.600000


 b) The proportions of the class counts are imbalanced: some classes have more than twice as many rows than others

## Question 3
Let's now consider the following pipeline:

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])

Evaluate the pipeline using stratified 10-fold cross-validation with the balanced-accuracy scoring metric to choose the correct statement in the list below.

You can use:

sklearn.model_selection.cross_validate to perform the cross-validation routine;

provide an integer 10 to the parameter cv of cross_validate to use the cross-validation with 10 folds;

provide the string "balanced_accuracy" to the parameter scoring of cross_validate.

In [7]:
from sklearn.model_selection import cross_validate

cv = cross_validate(model, data, target, 
                    cv = 10, scoring = 'balanced_accuracy')


cv = pd.DataFrame(cv)
cv

,fit_time,score_time,test_score
0,0.006210,0.006152,1.000000
1,0.003887,0.003669,1.000000
2,0.003896,0.004389,1.000000
3,0.003853,0.003721,0.918803
4,0.003753,0.003658,0.882540
5,0.003265,0.003314,0.952381
6,0.003552,0.003381,0.977778
7,0.003955,0.008611,0.930159
8,0.007402,0.006295,0.907937
9,0.006110,0.006250,0.952381


In [8]:
cv.test_score.mean()

0.9521978021978021

a) The average cross-validated test balanced accuracy of the above pipeline is between 0.9 and 1.0

## Question 4

Repeat the evaluation by setting the parameters in order to select the correct statements in the list below. We recall that you can use model.get_params() to list the parameters of the pipeline and use model.set_params(param_name=param_value) to update them.

Remember that one way to compare two models is comparing the cross-validation test scores of both models fold-to-fold, i.e. counting the number of folds where one model has a better test score than the other.

In [9]:
model.get_params()

{'memory': None,
 'steps': [('preprocessor', StandardScaler()),
  ('classifier', KNeighborsClassifier())],
 'verbose': False,
 'preprocessor': StandardScaler(),
 'classifier': KNeighborsClassifier(),
 'preprocessor__copy': True,
 'preprocessor__with_mean': True,
 'preprocessor__with_std': True,
 'classifier__algorithm': 'auto',
 'classifier__leaf_size': 30,
 'classifier__metric': 'minkowski',
 'classifier__metric_params': None,
 'classifier__n_jobs': None,
 'classifier__n_neighbors': 5,
 'classifier__p': 2,
 'classifier__weights': 'uniform'}

In [10]:
n_neighbors = [5, 51, 101]

neighbor_test = {}

for n_neig in n_neighbors:
    model.set_params(classifier__n_neighbors = n_neig)
    cv = cross_validate(model, data, target, 
                    cv = 10, scoring = 'balanced_accuracy')
    neighbor_test[n_neig] = cv['test_score']
    
result = pd.DataFrame(neighbor_test)
    
result

,5,51,101
0,1.000000,0.952381,0.857143
1,1.000000,0.977778,0.952381
2,1.000000,1.000000,0.944444
3,0.918803,0.863248,0.863248
4,0.882540,0.882540,0.834921
5,0.952381,0.952381,0.857143
6,0.977778,0.955556,0.834921
7,0.930159,0.952381,0.882540
8,0.907937,0.930159,0.834921
9,0.952381,0.952381,0.904762


In [11]:
model.set_params(preprocessor__with_mean = False,
                 preprocessor__with_std = False)

cv = cross_validate(model, data, target, 
                cv = 10, scoring = 'balanced_accuracy')

pd.DataFrame(cv)

,fit_time,score_time,test_score
0,0.005042,0.005636,0.618056
1,0.003879,0.004234,0.593162
2,0.003763,0.004188,0.574359
3,0.003485,0.004519,0.564103
4,0.003462,0.003904,0.588889
5,0.003562,0.005196,0.644444
6,0.003345,0.004100,0.622222
7,0.003473,0.004066,0.622222
8,0.003455,0.003915,0.644444
9,0.003752,0.004421,0.666667


b) Looking at the individual cross-validation scores, using a model with n_neighbors=5 is substantially better (at least 7 of the cross-validations scores are better) than a model with n_neighbors=101

c) Looking at the individual cross-validation scores, a 5 nearest neighbors using a StandardScaler is substantially better (at least 7 of the cross-validations scores are better) than a 5 nearest neighbors using the raw features (without scaling).

Another possible way of answering

It is possible to change the pipeline parameters and re-run a cross-validation with:

In [12]:
from sklearn.model_selection import cross_validate

model.set_params(preprocessor=StandardScaler(), classifier__n_neighbors=5)
cv_results_ss_5 = cross_validate(
    model, data, target, cv=10, scoring="balanced_accuracy"
)
cv_results_ss_5["test_score"].mean(), cv_results_ss_5["test_score"].std()


model.set_params(preprocessor=StandardScaler(), classifier__n_neighbors=51)
cv_results_ss_51 = cross_validate(
    model, data, target, cv=10, scoring="balanced_accuracy"
)
cv_results_ss_51["test_score"].mean(), cv_results_ss_51["test_score"].std()

(0.9418803418803419, 0.03890547525064432)

which gives slightly worse test scores but the difference is not necessarily significant: they overlap a lot. So given the definition of better, we can check the individual score for each fold and count how many times the 5-NN classifier is better than the 51-NN classifier. With some python code (you could have do it by visualizing the "test_score" columns as well), we obtain:

In [13]:
print(
    "5-NN is strictly better than 51-NN for "
    f"{sum(cv_results_ss_5['test_score'] > cv_results_ss_51['test_score'])}"
    " CV iterations out of 10."
  )

5-NN is strictly better than 51-NN for 4 CV iterations out of 10.


Here, 5-NN is strictly better than 51-NN only 4 times and thus we cannot conclude that it is substantially better.

We can repeat the same experiment for a 101-NN:

In [14]:
model.set_params(preprocessor=StandardScaler(), classifier__n_neighbors=101)
cv_results_ss_101 = cross_validate(
    model, data, target, cv=10, scoring="balanced_accuracy"
)
cv_results_ss_101["test_score"].mean(), cv_results_ss_101["test_score"].std()

(0.8766422466422465, 0.04161841544181347)

We observe that the average test accuracy of this last model seems to be substantially lower that the previous models. Let's check the number of CV folds where this is actually the case:

In [15]:
print(
    "5-NN is strictly better than 101-NN for "
    f"{sum(cv_results_ss_5['test_score'] > cv_results_ss_101['test_score'])}"
    "CV iterations out of 10."
  )

5-NN is strictly better than 101-NN for 10CV iterations out of 10.


In this case, we observe that 5-NN is always better.

We can disable the preprocessor by setting the preprocessor parameter to None (while resetting the number of neighbors to 5) as follows:

In [16]:
model.set_params(preprocessor=None, classifier__n_neighbors=5)
cv_results_none_5 = cross_validate(
    model, data, target, cv=10, scoring="balanced_accuracy"
)
cv_results_none_5["test_score"].mean(), cv_results_none_5["test_score"].std()

(0.7398382173382173, 0.08668489381180364)

This gives results with a mean balanced accuracy of ~0.74 which is much worse than the same result with preprocessing enabled. We can confirm that preprocessing the dataset lead to a substantially better model:

In [17]:
print(
    "NN with scaling is better NN without scaling for "
    f"{sum(cv_results_ss_5['test_score'] > cv_results_none_5['test_score'])}"
    "CV iterations out of 10."
  )

NN with scaling is better NN without scaling for 10CV iterations out of 10.


Here, the model with feature scaling is performing better 10 times over 10 than the model that does not preprocess the dataset.

## Question 5 
We will now study the impact of different preprocessors defined in the list below:

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer


all_preprocessors = [
    None,
    StandardScaler(),
    MinMaxScaler(),
    QuantileTransformer(n_quantiles=100),
    PowerTransformer(method="box-cox"),
]

The Box-Cox method is common preprocessing strategy for positive values. The other preprocessors work both for any kind of numerical features. If you are curious to read the details about those method, please feel free to read them up in the preprocessing chapter of the scikit-learn user guide but this is not required to answer the quiz questions.

Use sklearn.model_selection.GridSearchCV to study the impact of the choice of the preprocessor and the number of neighbors on the stratified 10-fold cross-validated balanced_accuracy metric. We want to study the n_neighbors in the range [5, 51, 101] and preprocessor in the range all_preprocessors.

Which of the following statements hold:

In [19]:
from sklearn.model_selection import GridSearchCV

model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])

param_grid = {"preprocessor": all_preprocessors,
              "classifier__n_neighbors": [5, 51, 101]}

grid_search = GridSearchCV(model, param_grid = param_grid, cv = 10,
                          scoring = 'balanced_accuracy')

grid_search.fit(data, target)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor', StandardScaler()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid={'classifier__n_neighbors': [5, 51, 101],
                         'preprocessor': [None, StandardScaler(),
                                          MinMaxScaler(),
                                          QuantileTransformer(n_quantiles=100),
                                          PowerTransformer(method='box-cox')]},
             scoring='balanced_accuracy')

In [20]:
results = pd.DataFrame(grid_search.cv_results_).sort_values(
    by="rank_test_score", ascending=True
)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__n_neighbors,param_preprocessor,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
1,0.003468,0.000326,0.003483,0.000395,5,StandardScaler(),"{'classifier__n_neighbors': 5, 'preprocessor':...",1.000000,1.000000,1.000000,0.918803,0.882540,0.952381,0.977778,0.930159,0.907937,0.952381,0.952198,0.039902,1
2,0.003551,0.000198,0.003681,0.000249,5,MinMaxScaler(),"{'classifier__n_neighbors': 5, 'preprocessor':...",1.000000,0.952381,1.000000,0.944444,0.882540,0.930159,0.955556,0.952381,0.907937,0.952381,0.947778,0.034268,2
3,0.004650,0.000186,0.003789,0.000173,5,QuantileTransformer(n_quantiles=100),"{'classifier__n_neighbors': 5, 'preprocessor':...",0.952381,0.926740,1.000000,0.918803,0.904762,1.000000,0.977778,0.930159,0.907937,0.952381,0.947094,0.033797,3
4,0.007142,0.000553,0.003553,0.000140,5,PowerTransformer(method='box-cox'),"{'classifier__n_neighbors': 5, 'preprocessor':...",1.000000,0.977778,1.000000,0.863248,0.882540,0.952381,0.955556,0.930159,0.907937,1.000000,0.946960,0.047387,4
6,0.003424,0.000185,0.003694,0.000382,51,StandardScaler(),"{'classifier__n_neighbors': 51, 'preprocessor'...",0.952381,0.977778,1.000000,0.863248,0.882540,0.952381,0.955556,0.952381,0.930159,0.952381,0.941880,0.038905,5
8,0.004395,0.000135,0.003797,0.000112,51,QuantileTransformer(n_quantiles=100),"{'classifier__n_neighbors': 51, 'preprocessor'...",0.857143,0.952381,1.000000,0.863248,0.904762,0.904762,0.977778,0.930159,0.930159,0.952381,0.927277,0.043759,6
9,0.007110,0.000397,0.003869,0.000123,51,PowerTransformer(method='box-cox'),"{'classifier__n_neighbors': 51, 'preprocessor'...",0.904762,0.977778,1.000000,0.863248,0.834921,0.952381,0.907937,0.952381,0.930159,0.904762,0.922833,0.047883,7
7,0.003433,0.000209,0.003727,0.000170,51,MinMaxScaler(),"{'classifier__n_neighbors': 51, 'preprocessor'...",0.904762,0.952381,1.000000,0.863248,0.834921,0.952381,0.907937,0.952381,0.930159,0.904762,0.920293,0.045516,8
11,0.003426,0.000180,0.003746,0.000194,101,StandardScaler(),"{'classifier__n_neighbors': 101, 'preprocessor...",0.857143,0.952381,0.944444,0.863248,0.834921,0.857143,0.834921,0.882540,0.834921,0.904762,0.876642,0.041618,9
12,0.003167,0.000130,0.003664,0.000130,101,MinMaxScaler(),"{'classifier__n_neighbors': 101, 'preprocessor...",0.857143,0.857143,0.944444,0.863248,0.834921,0.857143,0.765079,0.904762,0.834921,0.904762,0.862357,0.046244,10


In [21]:
# convert the name of the preprocessor for later display
results["param_preprocessor"] = results["param_preprocessor"].apply(
    lambda x: x.__class__.__name__ if x is not None else "None"
)

results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__n_neighbors,param_preprocessor,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
1,0.003468,0.000326,0.003483,0.000395,5,StandardScaler,"{'classifier__n_neighbors': 5, 'preprocessor':...",1.000000,1.000000,1.000000,0.918803,0.882540,0.952381,0.977778,0.930159,0.907937,0.952381,0.952198,0.039902,1
2,0.003551,0.000198,0.003681,0.000249,5,MinMaxScaler,"{'classifier__n_neighbors': 5, 'preprocessor':...",1.000000,0.952381,1.000000,0.944444,0.882540,0.930159,0.955556,0.952381,0.907937,0.952381,0.947778,0.034268,2
3,0.004650,0.000186,0.003789,0.000173,5,QuantileTransformer,"{'classifier__n_neighbors': 5, 'preprocessor':...",0.952381,0.926740,1.000000,0.918803,0.904762,1.000000,0.977778,0.930159,0.907937,0.952381,0.947094,0.033797,3
4,0.007142,0.000553,0.003553,0.000140,5,PowerTransformer,"{'classifier__n_neighbors': 5, 'preprocessor':...",1.000000,0.977778,1.000000,0.863248,0.882540,0.952381,0.955556,0.930159,0.907937,1.000000,0.946960,0.047387,4
6,0.003424,0.000185,0.003694,0.000382,51,StandardScaler,"{'classifier__n_neighbors': 51, 'preprocessor'...",0.952381,0.977778,1.000000,0.863248,0.882540,0.952381,0.955556,0.952381,0.930159,0.952381,0.941880,0.038905,5
8,0.004395,0.000135,0.003797,0.000112,51,QuantileTransformer,"{'classifier__n_neighbors': 51, 'preprocessor'...",0.857143,0.952381,1.000000,0.863248,0.904762,0.904762,0.977778,0.930159,0.930159,0.952381,0.927277,0.043759,6
9,0.007110,0.000397,0.003869,0.000123,51,PowerTransformer,"{'classifier__n_neighbors': 51, 'preprocessor'...",0.904762,0.977778,1.000000,0.863248,0.834921,0.952381,0.907937,0.952381,0.930159,0.904762,0.922833,0.047883,7
7,0.003433,0.000209,0.003727,0.000170,51,MinMaxScaler,"{'classifier__n_neighbors': 51, 'preprocessor'...",0.904762,0.952381,1.000000,0.863248,0.834921,0.952381,0.907937,0.952381,0.930159,0.904762,0.920293,0.045516,8
11,0.003426,0.000180,0.003746,0.000194,101,StandardScaler,"{'classifier__n_neighbors': 101, 'preprocessor...",0.857143,0.952381,0.944444,0.863248,0.834921,0.857143,0.834921,0.882540,0.834921,0.904762,0.876642,0.041618,9
12,0.003167,0.000130,0.003664,0.000130,101,MinMaxScaler,"{'classifier__n_neighbors': 101, 'preprocessor...",0.857143,0.857143,0.944444,0.863248,0.834921,0.857143,0.765079,0.904762,0.834921,0.904762,0.862357,0.046244,10


 a) Looking at the individual cross-validation scores, the best ranked model using a StandardScaler is substantially better (at least 7 of the cross-validations scores are better) than using any other processor

In [22]:
# Now, we can check the validity of each statement.

# We see that the best model uses a StandardScaler while the subsequent three models are using the other preprocessors. We can thus check if StandardScaler is performing substantially better than the other models.

reference_model = results.iloc[0]
other_models = results.iloc[1:4]
cv_score_columns = results.columns[results.columns.str.startswith("split")]
for idx, other_model in other_models.iterrows():
    score_reference_model = reference_model[cv_score_columns]
    score_other_model = other_model[cv_score_columns]
    print(
        f"{reference_model['param_classifier__n_neighbors']}-NN with "
        f"{reference_model['param_preprocessor']} is strictly better than "
        f"{other_model['param_classifier__n_neighbors']}-NN with "
        f"{other_model['param_preprocessor']} for "
        f"{sum(score_reference_model > score_other_model)} CV iterations "
        f"out of 10."
    )

5-NN with StandardScaler is strictly better than 5-NN with MinMaxScaler for 3 CV iterations out of 10.
5-NN with StandardScaler is strictly better than 5-NN with QuantileTransformer for 2 CV iterations out of 10.
5-NN with StandardScaler is strictly better than 5-NN with PowerTransformer for 3 CV iterations out of 10.


Thus, a 5-NN model with a StandardScaler does not perform substantially better than the models that use alternative scaling strategies.

 b) Using any of the preprocessors has always a better ranking than using no processor, irrespective of the value of n_neighbors

Looking at the ranking, we see that not applying any preprocessing always lead to the worst results. The main reason that explains that removing the preprocessor leads to bad performance, is the fact that the input features have very different dynamic ranges when using the default units (grams and millimeters).

 c) Looking at the individual cross-validation scores, the model with n_neighbors=5 and StandardScaler is substantially better (at least 7 of the cross-validations scores are better) than the model with n_neighbors=51 and StandardScaler
 

In [23]:
import numpy as np

reference_model = results.iloc[0][cv_score_columns]
other_model = results.iloc[4][cv_score_columns]
print(
    f"5-NN with StandardScaler is strictly better 51-NN with StandardScaler for "
    f"{np.sum(reference_model.to_numpy() > other_model.to_numpy())} "
    "CV iterations out of 10."
)

5-NN with StandardScaler is strictly better 51-NN with StandardScaler for 4 CV iterations out of 10.


Thus, 5-NN is not substantially better 51-NN when the preprocessor is fixed to a StandardScaler.

d) Looking at the individual cross-validation scores, the model with n_neighbors=51 and StandardScaler is substantially better (at least 7 of the cross-validations scores are better) than the model with n_neighbors=101 and StandardScaler

In [24]:
reference_model = results.iloc[0][cv_score_columns]
other_model = results.iloc[8][cv_score_columns]
print(
    f"51-NN with StandardScaler is strictly better than 101-NN with StandardScaler for "
    f"{np.sum(reference_model.to_numpy() > other_model.to_numpy())} "
    "CV iterations out of 10."
)

51-NN with StandardScaler is strictly better than 101-NN with StandardScaler for 10 CV iterations out of 10.


In this case, we observe that 51-NN is substantially better than the 101-NN.

As usual, setting a too large value for n_neighbors cause under-fitting. Here the data is well structured and has not much noise: using low values for n_neighbors is as good or better than intermediate values as there is not much over-fitting possible.

## Question 6
Evaluate the generalization performance of the best models found in each fold using nested cross-validation. Set return_estimator=True and cv=10 for the outer loop. The scoring metric must be the balanced-accuracy.

The mean generalization performance is :

In [25]:
generalization_performance = cross_validate(grid_search, data, target, 
                                           cv=10, return_estimator= True,
                                           scoring = 'balanced_accuracy')

In [26]:
pd.DataFrame(generalization_performance)

,fit_time,score_time,estimator,test_score
0,1.282152,0.003937,"GridSearchCV(cv=10,\n estimator=Pi...",0.952381
1,1.313920,0.003378,"GridSearchCV(cv=10,\n estimator=Pi...",0.926740
2,1.251538,0.003213,"GridSearchCV(cv=10,\n estimator=Pi...",1.000000
3,1.227932,0.003410,"GridSearchCV(cv=10,\n estimator=Pi...",0.918803
4,1.264778,0.004033,"GridSearchCV(cv=10,\n estimator=Pi...",0.882540
5,1.254393,0.003596,"GridSearchCV(cv=10,\n estimator=Pi...",1.000000
6,1.242929,0.003474,"GridSearchCV(cv=10,\n estimator=Pi...",0.955556
7,1.234950,0.003217,"GridSearchCV(cv=10,\n estimator=Pi...",0.930159
8,1.210333,0.003256,"GridSearchCV(cv=10,\n estimator=Pi...",0.907937
9,1.406468,0.003681,"GridSearchCV(cv=10,\n estimator=Pi...",0.952381


In [27]:
test_scores = generalization_performance['test_score']

print("The mean generalization performance with the best hyperparamenters is: " f"{test_scores.mean():.3f} +/- {test_scores.std():.3f}")

The mean generalization performance with the best hyperparamenters is: 0.943 +/- 0.036


b) between 0.92 and 0.97

## Question 7 
Explore the set of best parameters that the different grid search models found in each fold of the outer cross-validation. Remember that you can access them with the best_params_ attribute of the estimator. Select all the statements that are true.

In [28]:
for cv_fold, estimator_in_fold in enumerate(generalization_performance["estimator"]):
    print(
        f"Best hyperparameters for fold #{cv_fold + 1}:\n"
        f"{estimator_in_fold.best_params_}"
    )

Best hyperparameters for fold #1:
{'classifier__n_neighbors': 5, 'preprocessor': QuantileTransformer(n_quantiles=100)}
Best hyperparameters for fold #2:
{'classifier__n_neighbors': 5, 'preprocessor': QuantileTransformer(n_quantiles=100)}
Best hyperparameters for fold #3:
{'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()}
Best hyperparameters for fold #4:
{'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()}
Best hyperparameters for fold #5:
{'classifier__n_neighbors': 5, 'preprocessor': MinMaxScaler()}
Best hyperparameters for fold #6:
{'classifier__n_neighbors': 5, 'preprocessor': QuantileTransformer(n_quantiles=100)}
Best hyperparameters for fold #7:
{'classifier__n_neighbors': 5, 'preprocessor': MinMaxScaler()}
Best hyperparameters for fold #8:
{'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()}
Best hyperparameters for fold #9:
{'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()}
Best hyperparameters for fold #10:
{'classifier__n_ne

 a) The tuned number of nearest neighbors is stable across all folds 
 
 d) The optimal scaler changes often across all folds 